![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DistilBertForQuestionAnswering.ipynb)

## Import ONNX DistilBertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DistilBertForQuestionAnswering` is only available since in `Spark NLP 5.1.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DistilBERT models trained/fine-tuned for question answering via `DistilBertForQuestionAnswering` or `TFDistilBertForQuestionAnswering`. These models are usually under `Question Answering` category and have `distilbert` in their labels
- Reference: [TFDistilBertForQuestionAnswering](https://huggingface.co/transformers/model_doc/distilbert#transformers.TFDistilBertForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=distilbert&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [3]:
!pip install -q --upgrade transformers[onnx]==4.29.1 optimum tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 17.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.7 MB

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- - We'll use [distilbert-base-cased-distilled-squad](https://huggingface.co/distilbert-base-cased-distilled-squad) model from HuggingFace as an example and load it as a `ORTModelForQuestionAnswering`, representing an ONNX model.

In [4]:
from optimum.onnxruntime import ORTModelForQuestionAnswering
import tensorflow as tf

MODEL_NAME = 'distilbert-base-cased-distilled-squad'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(ONNX_MODEL)

Framework not specified. Using pt to export to ONNX.


Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.0.1+cu118
/usr/local/lib/python3.10/dist-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Let's have a look inside these two directories and see what we are dealing with:

In [5]:
!ls -l {ONNX_MODEL}

total 255672
-rw-r--r-- 1 root root       569 Oct  5 19:45 config.json
-rw-r--r-- 1 root root 260905268 Oct  5 19:45 model.onnx
-rw-r--r-- 1 root root       125 Oct  5 19:45 special_tokens_map.json
-rw-r--r-- 1 root root       321 Oct  5 19:45 tokenizer_config.json
-rw-r--r-- 1 root root    668923 Oct  5 19:45 tokenizer.json
-rw-r--r-- 1 root root    213450 Oct  5 19:45 vocab.txt


- As you can see, we need to move `vocab.txt` from the tokenizer to `assets` folder which Spark NLP will look for

In [6]:
!mkdir {ONNX_MODEL}/assets

In [7]:
!mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/assets

Voila! We have our `vocab.txt` inside assets directory

In [8]:
!ls -lR {ONNX_MODEL}

onnx_models/distilbert-base-cased-distilled-squad:
total 255468
drwxr-xr-x 2 root root      4096 Oct  5 19:45 assets
-rw-r--r-- 1 root root       569 Oct  5 19:45 config.json
-rw-r--r-- 1 root root 260905268 Oct  5 19:45 model.onnx
-rw-r--r-- 1 root root       125 Oct  5 19:45 special_tokens_map.json
-rw-r--r-- 1 root root       321 Oct  5 19:45 tokenizer_config.json
-rw-r--r-- 1 root root    668923 Oct  5 19:45 tokenizer.json

onnx_models/distilbert-base-cased-distilled-squad/assets:
total 212
-rw-r--r-- 1 root root 213450 Oct  5 19:45 vocab.txt


## Import and Save DistilBertForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [9]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.1.2
setup Colab for PySpark 3.2.3 and Spark NLP 5.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.3/536.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 7.8 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [10]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()
print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `DistilBertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [11]:
from sparknlp.annotator import *
from sparknlp.base import *

spanClassifier = DistilBertForQuestionAnswering.loadSavedModel(
     ONNX_MODEL,
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [12]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [13]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your DistlBertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [14]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 254844
-rw-r--r-- 1 root root 260945257 Oct  5 19:48 distilbert_classification_onnx
drwxr-xr-x 3 root root      4096 Oct  5 19:47 fields
drwxr-xr-x 2 root root      4096 Oct  5 19:47 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [15]:
document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = DistilBertForQuestionAnswering.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([["What's my name?", "My name is Clara and I live in Berkeley."]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[Clara]|
+-------+



That's it! You can now go wild and use hundreds of `DistilBertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
